In [2]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址498.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}

In [3]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
    res = requests.get('http://medicalxpress.com/', headers=headers_pc, proxies = randomproxy,)
    # 将cookieJar数据转化为字典，
    cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

In [4]:
def get_article_url_list(url, headers_pc, randomproxy, cookie_dict, term, i):
    article_result = []
    try:
        requests.adapters.DEFAULT_RETRIES = 20 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, cookies=cookie_dict, proxies = randomproxy, timeout=10)
        r.encoding = 'utf-8'
        html = r.text
#         if os.path.exists(f"D:/Intern_Project/12_medicalxpress/列表页/{term}_{i}列表页.htm"):   
#             pass
        with open(f'D:/Intern_Project/12_medicalxpress/列表页/{term}_{i}列表页.html', 'w+', encoding='utf-8') as f:
             f.write(html)
        
        size = round(os.path.getsize(f'D:/Intern_Project/12_medicalxpress/列表页/{term}_{i}列表页.html')/float(1024),2)
        if size<2.0:
            time.sleep(random.randint(5,10))
            headers_pc, randomproxy, cookie_dict = changeparams()
            get_article_url_list(url, headers_pc, randomproxy, cookie_dict, term, i)
            mis_article_list.append(url)


        selector = Selector(text=html)
        titleUrl_list = selector.xpath(
            "//div[@class='sorted-article-content d-flex flex-column ie-flex-1']/h2/a")
        #print(len(termUrl_list))
        for titleUrl in titleUrl_list:
            tmp_url = titleUrl.xpath('./@href').extract()[0]
            tmp_title = titleUrl.xpath('.//text()').extract()[0]
            # print(term)
            tmp = {
                'title': tmp_title,
                'url': tmp_url
            }
            article_result.append(tmp)
            # time.sleep(1)
        # pbar.update(1)
    except Exception as e:
        print(e)
        time.sleep(random.randint(3,5))
        headers_pc, randomproxy, cookie_dict = changeparams()
        get_article_url_list(url, headers_pc, randomproxy, cookie_dict, term, i)
        mis_article_list.append(url)
    return article_result

In [5]:
def get_pagenum(url, headers_pc, randomproxy, cookie_dict):
    try:
        requests.adapters.DEFAULT_RETRIES = 20 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, cookies=cookie_dict, proxies = randomproxy, timeout=10)
        r.encoding = 'utf-8'
        html = r.text

        selector = Selector(text=html)
        pagenum = selector.xpath(
            "//div[@class='pagination-view mr-4']/span/text()").extract()[0]
        pagenum = int(pagenum)
    except Exception as e:
        print(e)
        time.sleep(random.randint(3,5))
        headers_pc, randomproxy, cookie_dict = changeparams()
        get_pagenum(url, headers_pc, randomproxy, cookie_dict)
    return pagenum

In [6]:
def task_monitor(task_list, mt_pool, results):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
                results.extend(result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [7]:
with open(f'D:/Intern_Project/12_medicalxpress/术语链接列表0_199.json', 'r', encoding='utf-8') as f:
    results = json.load(f)

In [8]:
mis_item_list = []
mis_article_list = []

all_craw_tasks = []
article_results = []

headers_pc, randomproxy, cookie_dict = changeparams()

# pbar = tqdm(total=len(results))
with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for item in results:
        pagenum = 0
        headers_pc, randomproxy, cookie_dict = changeparams()
        start_url = item['url'] + 'sort/date/all/'
        start_url.replace('https','http')
        term = item['term']
        try:
            pagenum = get_pagenum(start_url, headers_pc, randomproxy, cookie_dict)
        except Exception as e:
            print(e)
            mis_item_list.append(item)
        for i in range(pagenum):
            page_url = item['url'] + 'sort/date/all/' + f'page{i}.html'
            page_url.replace('https','http')
            tmp_task = mt_pool.submit(get_article_url_list, page_url, headers_pc, randomproxy, cookie_dict, term , i)
            all_craw_tasks.append(tmp_task)
            if i % 5 == 0:
                time.sleep(random.randint(3,5))
                headers_pc, randomproxy, cookie_dict = changeparams()
        # pbar.update(1)
    
    task_monitor(all_craw_tasks, mt_pool, article_results)
    
    with open(f'D:/Intern_Project/12_medicalxpress/文章链接列表.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(article_results, indent=2, ensure_ascii=False))

ConnectionError: HTTPConnectionPool(host='medicalxpress.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020F13139550>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

In [5]:
results = []
with open(f'D:/Intern_Project/12_medicalxpress/medicalxpress_article_info/medicalxpress_article_info.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        dic = json.loads(line)
        results.append(dic)

In [6]:
print(len(results))

57226


In [ ]:
if "Smog may shorten life span for those with heart failure" in 